In [ ]:
!pip install swing
!pip install gymnasium[box2d]
!pip install --upgrade moviepy
!pip install stable-baselines3
!sudo apt-get update
!sudo apt-get install -y swig
!sudo apt-get install -y python3-dev
!pip install "gymnasium[box2d]"

  Preparing metadata (setup.py) ... done
  Created wheel for swing: filename=swing-0.0.4-py3-none-any.whl size=3240 sha256=8faff69214bef650f1f6070c9fc94be411342d8f0d99c1979d49417f6e2577fe
  Stored in directory: /root/.cache/pip/wheels/32/62/99/7f2bbbf16199481ca1f828144c44068d3b0300c6ba89844052
Successfully built swing
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import gymnasium as gym
# import numpy as np
# import cv2
# from gymnasium.wrappers import TransformObservation
# from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# from stable_baselines3.common.monitor import Monitor
# from gymnasium.spaces import Box

# # Grayscale + Resize function to preprocess the observation
# # The original observation from CarRacing-v3 is (96, 96, 3)
# def preprocess_obs(obs):
#     # Convert the observation to grayscale
#     gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)  # Shape (96, 96)

#     # Resize the grayscale image to a smaller size
#     resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)  # Shape (84, 84)

#     # Add a channel dimension to the resized image and ensure the data type is uint8
#     # This results in a shape of (84, 84, 1), which is a common format for CNNs
#     return np.expand_dims(resized, axis=-1).astype(np.uint8)

# # Define the new observation space to match the preprocessed observation
# new_obs_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

# # Factory function to create the wrapped environment
# def make_env():
#     # Create the CarRacing-v3 environment in rgb_array mode
#     env = gym.make("CarRacing-v3", render_mode="rgb_array")

#     # Wrap the environment to transform the observation.
#     # The key change is using 'func=preprocess_obs' instead of 'f=preprocess_obs'.
#     # This is the correct keyword argument for the TransformObservation wrapper.
#     env = TransformObservation(env, func=preprocess_obs, observation_space=new_obs_space)

#     # Monitor the environment to track metrics like episode rewards
#     env = Monitor(env)
#     return env

# # Create a vectorized environment from the factory function
# # DummyVecEnv is a simple wrapper for a list of environments
# env = DummyVecEnv([make_env])

# # Stack 4 frames together to capture motion over time
# # The channels_order="last" argument is important for compatibility with frameworks
# # like Stable-Baselines3, which expect the channel dimension to be last (e.g., H, W, C)
# env = VecFrameStack(env, n_stack=4, channels_order="last")

# # You can now use this 'env' object for training your Stable-Baselines3 agent
# print("Environment setup complete. The vectorized and frame-stacked environment is ready for training.")
# print(f"Observation space after frame stacking: {env.observation_space.shape}")
# print(f"Action space: {env.action_space}")


In [ ]:
# import os
# from stable_baselines3 import PPO
# from stable_baselines3.common.evaluation import evaluate_policy
# from stable_baselines3.common.callbacks import CheckpointCallback

# # Ensure log directory exists
# os.makedirs("./logs", exist_ok=True)

# # Initialize the model with CnnPolicy and your custom-preprocessed env
# model = PPO("CnnPolicy", env, device="cuda")

# # Total training timesteps
# total_timesteps = 1_000_000

# # Checkpoint callback
# checkpoint_callback = CheckpointCallback(
#     save_freq=100_000,
#     save_path='./logs/',
#     name_prefix='ppo_carracing'
# )

# # Train the model
# print("🚗 Starting model training...")
# model.learn(total_timesteps=total_timesteps, callback=checkpoint_callback, progress_bar=True)
# print("✅ Training finished.")

# # Evaluate the trained policy
# n_eval_episodes = 10
# print(f"📊 Starting evaluation over {n_eval_episodes} episodes...")
# mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes)
# print(f"🎯 Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")

# # Save final model
# model.save("./logs/ppo_carracing_final")
# print("✅ Final model saved to logs/ppo_carracing_final")

# # Close the environment
# env.close()


In [1]:
import os
import gymnasium as gym
import numpy as np
import cv2
from gymnasium.wrappers import TransformObservation
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback
from gymnasium.spaces import Box

# ===============================================
# Environment Setup
# ===============================================

# Grayscale + Resize function to preprocess the observation
def preprocess_obs(obs):
    """
    Converts a raw RGB observation from the environment to a single-channel,
    resized grayscale image.
    """
    gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
    return np.expand_dims(resized, axis=-1).astype(np.uint8)

# Define the new observation space
new_obs_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

# Factory function to create the wrapped environment
def make_env():
    """
    Creates and returns a single instance of the CarRacing-v3 environment
    with the necessary wrappers for preprocessing and monitoring.
    """
    env = gym.make("CarRacing-v3", render_mode="rgb_array", continuous=True)
    env = TransformObservation(env, func=preprocess_obs, observation_space=new_obs_space)
    env = Monitor(env)
    return env

# Create a vectorized environment from the factory function
env = DummyVecEnv([make_env])

# Stack 4 frames together to capture motion over time
env = VecFrameStack(env, n_stack=4, channels_order="last")


# ===============================================
# Load and Continue Training the Model
# ===============================================

# Define the path to the model checkpoint you want to continue from
              # "/content/logs/ppo_carracing_500000_steps.zip"
model_path = "log\ppo_carracing_500000_steps.zip" #"/content/logs/ppo_carracing_500000_steps.zip"

print(f"🔄 Loading model from: {model_path}")
try:
    # Load the trained model. Stable-Baselines3 automatically handles continuing
    # the training count from the loaded model's internal state.
    model = PPO.load(model_path, env=env, device="cuda",verbose=0)
    print("✅ Model loaded successfully!")

    # Define the number of *additional* timesteps you want to train for.
    # If the previous training ran for 400,000 steps and you want a total of 1,000,000,
    # you need to train for an additional 600,000 steps.
    additional_timesteps = 500_000

    print(f"🚗 Resuming training for an additional {additional_timesteps} timesteps...")

    # Checkpoint callback to save the model periodically during the new training run
    checkpoint_callback = CheckpointCallback(
        save_freq=100000,
        save_path='./logs/',
        name_prefix='ppo_carracing'
    )

    # Continue the training process. The `learn` method will automatically
    # start from the number of timesteps saved in the model's internal state.
    model.learn(total_timesteps=additional_timesteps, callback=checkpoint_callback, progress_bar=True)
    print("✅ Training finished.")

    # Evaluate the final policy after the additional training
    n_eval_episodes = 10
    print(f"📊 Starting final evaluation over {n_eval_episodes} episodes...")
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes)
    print(f"🎯 Mean Reward after training up to 1,000,000 steps: {mean_reward:.2f} ± {std_reward:.2f}")

    # Save the final model
    model.save("./logs/ppo_carracing_final_continued")
    print("✅ Final model saved to logs/ppo_carracing_final_continued")

except FileNotFoundError:
    print(f"❌ Error: Model file not found at {model_path}. Please check the path and try again.")

# Close the environment
env.close()


c:\Users\user\anaconda3\envs\ai_env\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


🔄 Loading model from: log\ppo_carracing_500000_steps.zip
Wrapping the env in a VecTransposeImage.


Output()

✅ Model loaded successfully!
🚗 Resuming training for an additional 500000 timesteps...


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -25.2    |
| time/              |          |
|    fps             | 37       |
|    iterations      | 1        |
|    time_elapsed    | 54       |
|    total_timesteps | 2048     |
---------------------------------


...

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1e+03    |
|    ep_rew_mean          | 7.38     |
| time/                   |          |
|    fps                  | 35       |
|    iterations           | 242      |
|    time_elapsed         | 13958    |
|    total_timesteps      | 495616   |
| train/                  |          |
|    approx_kl            | 2.25941  |
|    clip_fraction        | 0.646    |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.474   |
|    explained_variance   | 0.907    |
|    learning_rate        | 0.0003   |
|    loss                 | 0.112    |
|    n_updates            | 4850     |
|    policy_gradient_loss | -0.00503 |
|    std                  | 0.29     |
|    value_loss           | 0.548    |
--------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1e+03     |
|    ep_rew_mean          | 8.45      |
| time/                   |           |
|    fps                  | 35        |
|    iterations           | 243       |
|    time_elapsed         | 14024     |
|    total_timesteps      | 497664    |
| train/                  |           |
|    approx_kl            | 1.0439696 |
|    clip_fraction        | 0.719     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.467    |
|    explained_variance   | 0.955     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.00441   |
|    n_updates            | 4860      |
|    policy_gradient_loss | -0.00566  |
|    std                  | 0.291     |
|    value_loss           | 0.424     |
---------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1e+03      |
|    ep_rew_mean          | 10.1       |
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 244        |
|    time_elapsed         | 14079      |
|    total_timesteps      | 499712     |
| train/                  |            |
|    approx_kl            | 0.23040095 |
|    clip_fraction        | 0.626      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.462     |
|    explained_variance   | 0.971      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0405     |
|    n_updates            | 4870       |
|    policy_gradient_loss | -0.0163    |
|    std                  | 0.291      |
|    value_loss           | 0.538      |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1e+03      |
|    ep_rew_mean          | 12.2       |
| time/                   |            |
|    fps                  | 35         |
|    iterations           | 245        |
|    time_elapsed         | 14128      |
|    total_timesteps      | 501760     |
| train/                  |            |
|    approx_kl            | 0.31504968 |
|    clip_fraction        | 0.603      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.452     |
|    explained_variance   | 0.976      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0156     |
|    n_updates            | 4880       |
|    policy_gradient_loss | -0.0186    |
|    std                  | 0.289      |
|    value_loss           | 0.448      |
----------------------------------------


✅ Training finished.
📊 Starting final evaluation over 10 episodes...
🎯 Mean Reward after training up to 1,000,000 steps: -29.68 ± 13.04
✅ Final model saved to logs/ppo_carracing_final_continued


In [4]:
import os

# Define path in Google Drive
working_dir = "./"#"/content/drive/MyDrive/husob/06 ReinForcement Learning/005"
os.makedirs(working_dir, exist_ok=True)  # Ensure the directory exists
total_timesteps=1_000_000
# Use a proper, typo-free name
model_file_name = f"ppo_car_racing_{total_timesteps}"
model_file_path = os.path.join(working_dir, model_file_name)

# Save the model
model.save(model_file_path)

print(f"✅ Model successfully saved to:\n{model_file_path}")

✅ Model successfully saved to:
./ppo_car_racing_1000000
